In [3]:
import csv
from datetime import datetime
from datetime import timedelta
import os
import glob
from obspy import read, UTCDateTime, Stream
from tqdm import tqdm

In [4]:
path_data_out = './AA/202021AAS_BandPass1-15/'
path_data_in = './AA/202021X/'
path_handpicks = './AA/AA_hand_picks.csv'
path_phasenet_input_AAS = './AA/AAS_phasenet_input.csv'
path_all_hand_p = './AA/AAS_all_hand_p_picks.csv'
path_all_hand_s = './AA/AAS_all_hand_s_picks.csv'

In [5]:
path_project_home = '/Users/utkuunal/Desktop/Projects/ProjectA'
os.chdir('{}'.format(path_project_home))

In [6]:
ppicks = []
spicks = []
with open(path_handpicks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[2] == 'P':
            ppicks.append(row)
        elif row[2] == 'S':
            spicks.append(row)
print(len(ppicks))
print(ppicks[0:10])
print(len(spicks))
print(spicks[0:10])

7095
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '2020-05-14T01:17:30.54', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '2020-05-14T01:17:30.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:32.42', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:36.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:37.61', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:39.06', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '2020-05-14T01:17:39.94', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:42.80', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '2020-05-14T01:17:43.48', 'P', '20200514_0117']

In [7]:
isExist = os.path.exists(path_data_out)
if not isExist:
   os.makedirs(path_data_out)
   print("OK! The new output directory is created!")
else:
    print('CAUTION! Output directory exists')

OK! The new output directory is created!


In [8]:
isExist = os.path.exists(path_data_in)
if not isExist:
   print("ERROR! Input directory does not exist")
else:
    print('OK! Input directory exists')

OK! Input directory exists


In [9]:
i = 0
ppicks_corrected = []
for ppick in ppicks:
    sec = int((ppick[1][17:19]))
    min = int((ppick[1][14:16]))
    if  sec > 59:
        error_time = ppick[1][:16] + ':' + str(sec%60) + (ppick[1][19:22])
        new_time = datetime.strptime(error_time, '%Y-%m-%dT%H:%M:%S.%f') + timedelta(seconds = 60)
        ppicks_corrected.append([ppick[0], new_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-4], ppick[2], ppick[3]])
        i += 1
    else:
        ppicks_corrected.append([ppick[0], ppick[1], ppick[2], ppick[3]])
print('Corrected P picks: ', i)
print(len(ppicks_corrected))
print(ppicks_corrected[0:10])

Corrected P picks:  0
7095
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '2020-05-14T01:17:30.54', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '2020-05-14T01:17:30.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:32.42', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:36.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:37.61', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:39.06', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '2020-05-14T01:17:39.94', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:42.80', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '2020-05-14T01:17:43.48',

In [10]:
i = 0
spicks_corrected = []
for spick in spicks:
    sec = int((spick[1][17:19]))
    min = int((spick[1][14:16]))
    if  sec > 59:
        error_time = spick[1][:16] + ':' + str(sec%60) + (spick[1][19:22])
        new_time = datetime.strptime(error_time, '%Y-%m-%dT%H:%M:%S.%f') + timedelta(seconds = 60)
        spicks_corrected.append([spick[0], new_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-4], spick[2], spick[3]])
        i += 1
    else:
        spicks_corrected.append([spick[0], spick[1], spick[2], spick[3]])
print('Corrected S picks: ', i)
print(len(spicks_corrected))
print(spicks_corrected[0:10])

Corrected S picks:  593
3689
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:39.00', 'S', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:46.04', 'S', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:47.05', 'S', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:50.07', 'S', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:56.07', 'S', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '2020-05-14T01:17:58.04', 'S', '20200514_0117'], ['20200514_021748--39.59-26.09--Ke-TUZLA-AYVACIK-(CANAKKALE).M=1.3/BOZC.BH', '2020-05-14T02:17:58.01', 'S', '20200514_0217'], ['20200514_021748--39.59-26.09--Ke-TUZLA-AYVACIK-(CANAKKALE).M=1.3/COMU.HH', '2020-05-14T02:18:08.05', 'S', '20200514_0217'], ['20200514_021748--39.59-26.09--Ke-TUZLA-AYVACIK-(CA

In [11]:
allpicks = []
for ppick in tqdm(ppicks_corrected):
    temps = 'NO'
    for spick in spicks_corrected:
        if ppick[0] == spick[0]:
            temps = spick[1]
    if temps != 'NO':
        a = datetime.strptime(ppick[1], '%Y-%m-%dT%H:%M:%S.%f')
        b = datetime.strptime(temps, '%Y-%m-%dT%H:%M:%S.%f')
        diff1 = a - b
        if (abs(diff1.total_seconds())) < 25:
            allpicks.append([ppick[0], ppick[1], temps, ppick[3]])
    else:
        allpicks.append([ppick[0], ppick[1], temps, ppick[3]])
print(len(allpicks))
print(allpicks[0:10])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7095/7095 [00:01<00:00, 6622.76it/s]

7091
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '2020-05-14T01:17:30.54', 'NO', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '2020-05-14T01:17:30.68', 'NO', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:32.42', '2020-05-14T01:17:39.00', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:36.68', '2020-05-14T01:17:46.04', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:37.61', '2020-05-14T01:17:47.05', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:39.06', '2020-05-14T01:17:50.07', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '2020-05-14T01:17:39.94', 'NO', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:42.80', '2020-05-14T01:17:56.07', '20200514_0117'], [

In [12]:
allp = []
alls = []
for allpick in allpicks:
    allp.append([allpick[0], allpick[1], 'P', allpick[3]])
    if allpick[2] != 'NO':
        alls.append([allpick[0], allpick[2], 'S', allpick[3]])
print(len(allp) - len(ppicks_corrected))
print(len(alls) - len(spicks_corrected))
print(len(allp))
print(allp[0:10])
print(len(alls))
print(alls[0:10])

-4
-4
7091
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '2020-05-14T01:17:30.54', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '2020-05-14T01:17:30.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:32.42', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:36.68', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:37.61', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:39.06', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '2020-05-14T01:17:39.94', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:42.80', 'P', '20200514_0117'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '2020-05-14T01:17:43.48', 'P', '20200514_

In [13]:
os.chdir('{}'.format(path_project_home))
for ppick in tqdm(allp):
    current_path = path_data_in + ppick[0].split('/')[0]
    next_path = path_data_out + ppick[0].split('/')[0]
    os.chdir('{}'.format(current_path))
    current_station = ppick[0].split('/')[1]
    current_seismographs = glob.glob('{}*'.format(current_station))
    for curr_seis in current_seismographs:
        os.chdir('{}'.format(path_project_home))
        os.chdir('{}'.format(current_path))
        st = read(curr_seis)
        if st[0].stats.sampling_rate != 100:
            st.resample(100)
        st.filter("bandpass", freqmin=1, freqmax=15)
        start_time = UTCDateTime(ppick[1]) 
        st_new = st.slice(start_time - 5, start_time + 25)
        os.chdir('{}'.format(path_project_home))
        isExist = os.path.exists(next_path)
        if not isExist:
            os.makedirs(next_path)
        os.chdir('{}'.format(next_path))
        st_new.write(curr_seis, format='SAC')
    os.chdir('{}'.format(path_project_home))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7091/7091 [00:53<00:00, 133.60it/s]


In [14]:
i = 0
with open(path_phasenet_input_AAS, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['fname'])
    for pick in allpicks:
        writer.writerow([path_data_out + pick[0] + '*'])
        i += 1
print(i)

7091


In [15]:
i = 0
with open(path_all_hand_p, 'w', newline='') as file:
    writer = csv.writer(file)
    for ppick in allp:
        writer.writerow(ppick)
        i += 1
print(i)

7091


In [16]:
i = 0
with open(path_all_hand_s, 'w', newline='') as file:
    writer = csv.writer(file)
    for spick in alls:
        writer.writerow(spick)
        i += 1
print(i)

3685
